## TensorBoard Session Lab: tf.estimator

In this lab, you will take the solution you created in Lab 02b and implement TensorBoard summaries.

#### Try: compare time for each step with vs. without Summary Histograms.

Helpful references:
- https://www.youtube.com/watch?reload=9&v=eBbEDRsCmv4

In [1]:
import tensorflow as tf
import time
import os
import numpy as np

### DO: Summary Histograms

Implement the following for both _conv and _dense functions:
- Summary Histogram for weights.
- Summary Histogram for biases.
- Summary Histogram for activations.

Optional: Writing summaries can slow down training, and it is best to run with minimal summaries and only increase logging as needed for troubleshooting.  Optionally, implement logging flag to turn on/off summaries.

In [2]:
# Convolution Block

def _conv(x,kernel,name,log=False):
    with tf.variable_scope(name):
        W = tf.get_variable(initializer=tf.truncated_normal(shape=kernel,stddev=0.01),name='W')
        b = tf.get_variable(initializer=tf.constant(0.0,shape=[kernel[3]]),name='b')
        conv = tf.nn.conv2d(x, W, strides=[1,1,1,1],padding='SAME')
        activation = tf.nn.relu(tf.add(conv,b))
        pool = tf.nn.max_pool(activation,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        if log==True:
            tf.summary.histogram("weights",W)
            tf.summary.histogram("biases",b)
            tf.summary.histogram("activations",activation)
        return pool

# Dense Block

def _dense(x,size_in,size_out,name,relu=False,log=False):
    with tf.variable_scope(name):
        flat = tf.reshape(x,[-1,size_in])
        W = tf.get_variable(initializer=tf.truncated_normal([size_in,size_out],stddev=0.1),name='W')
        b = tf.get_variable(initializer=tf.constant(0.0,shape=[size_out]),name='b')
        activation = tf.add(tf.matmul(flat,W),b)
        if relu==True:
            activation = tf.nn.relu(activation)
        if log==True:
            tf.summary.histogram("weights",W)
            tf.summary.histogram("biases",b)
            tf.summary.histogram("activations",activation)
        return activation

### DO: Summary Scalars

Implement in MODE=TRAIN:
- Summary Scalar for Accuracy.
- Summary Scalar for Learning Rate (if using exponential decay function).

In [3]:
def cnnmodel_fn(features, labels, mode, params):
    
    #### 1 INFERNCE MODEL
    
    input_layer = tf.reshape(features, [-1, 32, 32, 3])
    conv1 = _conv(input_layer,kernel=[5,5,3,128],name='conv1',log=params['log'])
    conv2 = _conv(conv1,kernel=[5,5,128,128],name='conv2',log=params['log'])
    conv3 = _conv(conv2,kernel=[3,3,128,256],name='conv3',log=params['log'])
    conv4 = _conv(conv3,kernel=[3,3,256,512],name='conv4',log=params['log'])
    dense = _dense(conv4,size_in=2*2*512,size_out=params['dense_units'],
                   name='Dense',relu=True,log=params['log'])
    if mode==tf.estimator.ModeKeys.TRAIN:
        dense = tf.nn.dropout(dense,params['drop_out'])
    logits = _dense(dense,size_in=params['dense_units'],
                    size_out=10,name='Output',relu=False,log=params['log'])
        
    #### 2 CALCULATIONS AND METRICS
    
    predictions = {"classes": tf.argmax(input=logits,axis=1),
                   "logits": logits,
                   "probabilities": tf.nn.softmax(logits,name='softmax')}
    export_outputs = {'predictions': tf.estimator.export.PredictOutput(predictions)}
    if (mode==tf.estimator.ModeKeys.TRAIN or mode==tf.estimator.ModeKeys.EVAL):
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
        accuracy = tf.metrics.accuracy(
            labels=labels, predictions=tf.argmax(logits,axis=1))
        metrics = {'accuracy':accuracy}
        
    #### 3 MODE = PREDICT
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions, export_outputs=export_outputs)

    #### 4 MODE = TRAIN

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(
            params['learning_rate'],tf.train.get_global_step(),
            decay_steps=100000,decay_rate=0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        tf.summary.scalar('learning_rate', learning_rate)
        tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op)
    
    #### 5 MODE = EVAL
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,eval_metric_ops=metrics)

No Change.

In [15]:
def parse_tfrecord(example):
    feature={'idx'     : tf.FixedLenFeature((), tf.int64),
             'label'   : tf.FixedLenFeature((), tf.int64),
             'image'   : tf.FixedLenFeature((), tf.string, default_value="")}
    parsed = tf.parse_single_example(example, feature)
    image = tf.decode_raw(parsed['image'],tf.float64)
    image = tf.cast(image,tf.float32)
    image = tf.reshape(image,[32,32,3])
    return image, parsed['label']


No Change.

In [16]:
def image_scaling(x):
    return tf.image.per_image_standardization(x)

def distort(x):
    x = tf.image.resize_image_with_crop_or_pad(x, 40, 40)
    x = tf.random_crop(x, [32, 32, 3])
    x = tf.image.random_flip_left_right(x)
    return x

No Change.

In [17]:
def dataset_input_fn(params):
    dataset = tf.data.TFRecordDataset(
        params['filenames'],num_parallel_reads=params['threads'])
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=params['threads'])
    dataset = dataset.map(lambda x,y: (image_scaling(x),y),num_parallel_calls=params['threads'])
    if params['mode']==tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.map(lambda x,y: (distort(x),y),num_parallel_calls=params['threads'])
        dataset = dataset.shuffle(buffer_size=params['shuffle_buff'])
    dataset = dataset.repeat()
    dataset = dataset.batch(params['batch'])
    dataset = dataset.prefetch(8*params['batch'])
    return dataset

### DO: Logging Switch Parameter

Implement logging parameter to turn on/off histogram logging.

In [26]:
model_params  = {'drop_out'      : 0.2,
                 'dense_units'   : 1024,
                 'learning_rate' : 1e-3,
                 'log'           : False
                }

No Change.

In [27]:
config = tf.estimator.RunConfig(save_checkpoints_secs = 300,keep_checkpoint_max = 5)

In [28]:
#Set model_fn
model_fn = cnnmodel_fn

No Change.

In [29]:
name = 'cnn_model/cnn_model_'
name = name + 'dense' + str(model_params['dense_units']) + '_'
name = name + 'drop' + str(model_params['drop_out']) + '_'
name = name + 'lr' + str(model_params['learning_rate']) + '_'
name = name + time.strftime("%Y%m%d%H%M%S")
model_dir  = os.path.join('./',name)

print(model_dir)

./cnn_model/cnn_model_dense1024_drop0.2_lr0.001_20180716033729


No Change.

In [30]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,model_dir=model_dir,params=model_params,config=config)

INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f647c06c828>, '_task_type': 'worker', '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_task_id': 0, '_master': '', '_save_checkpoints_steps': None, '_is_chief': True, '_service': None, '_save_checkpoints_secs': 300, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_global_id_in_cluster': 0, '_model_dir': './cnn_model/cnn_model_dense1024_drop0.2_lr0.001_20180716033729', '_train_distribute': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_session_config': None, '_tf_random_seed': None}


No Change.

In [31]:
train_files = !ls ./data/cifar10_data_00*.tfrecords
val_files   = !ls ./data/cifar10_data_01*.tfrecords

train_params = {'filenames'    : train_files,
                'mode'         : tf.estimator.ModeKeys.TRAIN,
                'threads'      : 16,
                'shuffle_buff' : 100000,
                'batch'        : 100
               }

eval_params  = {'filenames'    : val_files,
                'mode'         : tf.estimator.ModeKeys.EVAL,
                'threads'      : 8,
                'batch'        : 200
               }

No Change.

In [32]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda: dataset_input_fn(train_params),max_steps=20000)
eval_spec  = tf.estimator.EvalSpec(input_fn=lambda: dataset_input_fn(eval_params),steps=10,throttle_secs=60)

No Change.

In [33]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./cnn_model/cnn_model_dense1024_drop0.2_lr0.001_20180716033729/model.ckpt.
INFO:tensorflow:step = 0, loss = 2.310493
INFO:tensorflow:global_step/sec: 71.3971
INFO:tensorflow:step = 100, loss = 2.2961326 (1.403 sec)
INFO:tensorflow:global_step/sec: 81.6835
INFO:tensorflow:step = 200, loss = 2.2942154 (1.226 sec)
INFO:tensorflow:global_step/sec: 81.4484
INFO:tensorflow:step = 300, loss = 2.2960572 (1.226 sec)
INFO:tensorflow:global_step/sec: 80.0307
INFO:tensorflow:step = 400, loss = 2.2899125 (1.249 s

KeyboardInterrupt: 